In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pii-data-detection-ckpt-ds/submission.csv
/kaggle/input/pii-data-detection-ckpt-ds/model.weights.h5
/kaggle/input/deberta_v3/keras/deberta_v3_small_en/2/config.json
/kaggle/input/deberta_v3/keras/deberta_v3_small_en/2/tokenizer.json
/kaggle/input/deberta_v3/keras/deberta_v3_small_en/2/metadata.json
/kaggle/input/deberta_v3/keras/deberta_v3_small_en/2/model.weights.h5
/kaggle/input/deberta_v3/keras/deberta_v3_small_en/2/assets/tokenizer/vocabulary.spm
/kaggle/input/kerasv3-lib-ds/tensorflow-2.15.0.post1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
/kaggle/input/kerasv3-lib-ds/keras_cv-0.8.1-py3-none-any.whl
/kaggle/input/kerasv3-lib-ds/keras_cv-0.8.2-py3-none-any.whl
/kaggle/input/kerasv3-lib-ds/keras-3.0.2-py3-none-any.whl
/kaggle/input/kerasv3-lib-ds/keras-3.0.4-py3-none-any.whl
/kaggle/input/kerasv3-lib-ds/tensorflow_io-0.35.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
/kaggle/input/pii-detection-removal-from-educational-data/sample_submiss

In [2]:
!pip install /kaggle/input/kerasv3-lib-ds/tensorflow-2.15.0.post1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install /kaggle/input/kerasv3-lib-ds/keras-3.0.4-py3-none-any.whl

Processing /kaggle/input/kerasv3-lib-ds/tensorflow-2.15.0.post1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.9 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: keras
    Found existing installation: keras 3.2.1
    Uninstalling keras-3.2.1:
      Successfully uninstalled keras-3.2.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.0
    Uninstalling tensorflow-2.15.0:
      Successfully uninstalled tensorflow-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
Processing /kaggle/input/kerasv3-lib-ds/keras-3.0.4-py3-none-any.whl
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0

In [3]:
import os
os.environ["KERAS_BACKEND"] = "jax"

import keras 
import keras_nlp
from keras import ops
import tensorflow as tf

import json
import numpy as np
import pandas as pd 
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split


import plotly.graph_objects as go
import plotly.express as px

2024-05-17 05:36:40.533401: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-17 05:36:40.533464: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-17 05:36:40.534892: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
print('tf',tf.__version__)
print('ks',keras.__version__)
print('ks_np',keras_nlp.__version__)

tf 2.15.0
ks 3.0.4
ks_np 0.9.3


In [5]:
class CFG:
    seed = 42
    preset = "deberta_v3_small_en" # name of pretrained backbone
    train_seq_len = 1024 # max size of input sequence for training
    train_batch_size = 2 * 8 # size of the input batch in training, x 2 as two GPUs
    infer_seq_len = 2000 # max size of input sequence for inference
    infer_batch_size = 2 * 2 # size of the input batch in inference, x 2 as two GPUs
    epochs = 6 # number of epochs to train
    lr_mode = "exp" # lr scheduler mode from one of "cos", "step", "exp"
    
    labels = ["B-EMAIL", "B-ID_NUM", "B-NAME_STUDENT", "B-PHONE_NUM",
              "B-STREET_ADDRESS", "B-URL_PERSONAL", "B-USERNAME",
              "I-ID_NUM", "I-NAME_STUDENT", "I-PHONE_NUM",
              "I-STREET_ADDRESS","I-URL_PERSONAL","O"]
    id2label = dict(enumerate(labels)) # integer label to BIO format label mapping
    label2id = {v:k for k,v in id2label.items()} # BIO format label to integer label mapping
    num_labels = len(labels) # number of PII (NER) tags
    
    train = True # whether to train or use already trained ckpt

In [6]:
keras.utils.set_random_seed(CFG.seed)

In [7]:
devices = keras.distribution.list_devices() #获取设备列表
print(devices)

if len(devices)>1:
    data_parallel = keras.distribution.DataParallel(devices=devices) #根据gpu数计算数据并行数量
    keras.distribution.set_distribution(data_parallel) #设置数据并行数量

['gpu:0', 'gpu:1']


In [8]:
keras.mixed_precision.set_global_policy("mixed_float16")

In [9]:
BASE_PATH = "/kaggle/input/pii-detection-removal-from-educational-data"

In [10]:
data = json.load(open(f'{BASE_PATH}/train.json'))

In [11]:
data[0]

{'document': 7,
 'full_text': "Design Thinking for innovation reflexion-Avril 2021-Nathalie Sylla\n\nChallenge & selection\n\nThe tool I use to help all stakeholders finding their way through the complexity of a project is the  mind map.\n\nWhat exactly is a mind map? According to the definition of Buzan T. and Buzan B. (1999, Dessine-moi  l'intelligence. Paris: Les Éditions d'Organisation.), the mind map (or heuristic diagram) is a graphic  representation technique that follows the natural functioning of the mind and allows the brain's  potential to be released. Cf Annex1\n\nThis tool has many advantages:\n\n•  It is accessible to all and does not require significant material investment and can be done  quickly\n\n•  It is scalable\n\n•  It allows categorization and linking of information\n\n•  It can be applied to any type of situation: notetaking, problem solving, analysis, creation of  new ideas\n\n•  It is suitable for all people and is easy to learn\n\n•  It is fun and encourages

In [12]:
words = np.empty(len(data),dtype=object)
labels = np.empty(len(data),dtype=object)
words

array([None, None, None, ..., None, None, None], dtype=object)

In [13]:
for i,v in tqdm(enumerate(data),total=len(data)):
    words[i] = np.array(v["tokens"])
    labels[i] = np.array([CFG.label2id[label] for label in v["labels"]])
labels[:1]

  0%|          | 0/6807 [00:00<?, ?it/s]

array([array([12, 12, 12, 12, 12, 12, 12, 12, 12,  2,  8, 12, 12, 12, 12, 12, 12,
              12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
              12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
              12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
              12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
              12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
              12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
              12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
              12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
              12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
              12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
              12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
              12

In [14]:
labels.shape

(6807,)

In [15]:
all_labels = np.array([x  for label in labels for x in label])
unique_labels,label_counts = np.unique(all_labels,return_counts=True)

fig = go.Figure(data=go.Bar(x=CFG.labels,y=label_counts))
fig.update_layout(
    title="Label Distribution",
    xaxis_title = 'Labels',
    yaxis_title = 'Count',
    yaxis_type = "log",
)

fig.update_traces(text=label_counts,textposition="outside")
fig.show()

In [16]:
train_words,valid_words,train_labels,valid_labels = train_test_split(
    words,labels,test_size=0.2,random_state=CFG.seed
)

In [17]:
train_words[:1]

array([array(['Visualization', 'tool', 'to', 'control', 'Humidity', '&', 'Temp',
              'in', 'Spinning', 'Plant', '\n\n', 'Challenge', '&', 'Selection',
              '\n\n', 'I', 'am', 'Electronics', 'Engineer', '&', 'working', 'as',
              'Chief', 'Engineer', 'in', 'a', 'Spinning', 'plant', '.', 'We',
              'have', 'to', 'face', '\n\n', 'challenges', '&', 'resolve', 'by',
              'anonymous', 'way', '.', 'We', 'do', 'not', 'use', 'any', 'design',
              'tool', 'to', 'resolve', 'any', 'problem', '.', 'Now', 'as', 'I',
              'have', 'enrolled', 'in', 'Design', 'thinking', 'course', ',', 'I',
              'started', 'to', 'apply', 'the', '1st', 'tool', 'as',
              'Visualization', 'to', 'check', 'its', 'validity', 'whether', 'it',
              'works', 'or', 'bookish', 'knowledge', 'only', '.', 'We', 'are',
              'facing', 'a', 'chronic', 'problem', 'of', 'Humidity', '&',
              'Temperature', 'Variation', 'in', 'Spi

In [18]:
tokenizer = keras_nlp.models.DebertaV3Tokenizer.from_preset(CFG.preset)
tokenizer

Attaching 'tokenizer.json' from model 'keras/deberta_v3/keras/deberta_v3_small_en/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/deberta_v3/keras/deberta_v3_small_en/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/deberta_v3/keras/deberta_v3_small_en/2' to your Kaggle notebook...


<DebertaV3Tokenizer name=deberta_v3_tokenizer, built=False>

In [19]:
??tokenizer

Signature:   tokenizer(*args, **kwargs)
Type:        DebertaV3Tokenizer
String form: <DebertaV3Tokenizer name=deberta_v3_tokenizer, built=False>
File:        /opt/conda/lib/python3.10/site-packages/keras_nlp/src/models/deberta_v3/deberta_v3_tokenizer.py
Source:     
@keras_nlp_export("keras_nlp.models.DebertaV3Tokenizer")
class DebertaV3Tokenizer(SentencePieceTokenizer):
    """DeBERTa tokenizer layer based on SentencePiece.

    This tokenizer class will tokenize raw strings into integer sequences and
    is based on `keras_nlp.tokenizers.SentencePieceTokenizer`. Unlike the
    underlying tokenizer, it will check for all special tokens needed by
    DeBERTa models and provides a `from_preset()` method to automatically
    download a matching vocabulary for a DeBERTa preset.

    This tokenizer does not provide truncation or padding of inputs. It can be
    combined with a `keras_nlp.models.DebertaV3Preprocessor` layer for input
    packing.

    If input is a batch of strings (rank > 

In [20]:
packer = keras_nlp.layers.MultiSegmentPacker(
    start_value=tokenizer.cls_token_id,
    end_value=tokenizer.sep_token_id,
    sequence_length=10
)

In [21]:
packer

<MultiSegmentPacker name=multi_segment_packer, built=False>

In [22]:
sample_words = words[0][:5]
sample_words

array(['Design', 'Thinking', 'for', 'innovation', 'reflexion'],
      dtype='<U16')

In [23]:
sample_token_int = [
    token.tolist() for word in sample_words for token in tokenizer(word)
]
sample_token_str = [tokenizer.id_to_token(token) for token in sample_token_int]

print("words        :", sample_words.tolist())
print("tokens (str) :", sample_token_str)
print("tokens (int) :", sample_token_int)

words        : ['Design', 'Thinking', 'for', 'innovation', 'reflexion']
tokens (str) : ['▁Design', '▁Thinking', '▁for', '▁innovation', '▁reflex', 'ion']
tokens (int) : [2169, 12103, 270, 3513, 28310, 4593]


In [24]:
padded_sample_tokens_int = packer(np.array(sample_token_int))[0].tolist()
padded_sample_tokens_str = [
    tokenizer.id_to_token(token) for token in padded_sample_tokens_int
]

print("tokens (str)        :", sample_token_str)
print("padded tokens (str) :", padded_sample_tokens_str, "\n")

print("tokens (int)        :", sample_token_int)
print("padded tokens (int) :", padded_sample_tokens_int)

tokens (str)        : ['▁Design', '▁Thinking', '▁for', '▁innovation', '▁reflex', 'ion']
padded tokens (str) : ['[CLS]', '▁Design', '▁Thinking', '▁for', '▁innovation', '▁reflex', 'ion', '[SEP]', '[PAD]', '[PAD]'] 

tokens (int)        : [2169, 12103, 270, 3513, 28310, 4593]
padded tokens (int) : [1, 2169, 12103, 270, 3513, 28310, 4593, 2, 0, 0]


/opt/conda/lib/python3.10/site-packages/keras/src/backend/jax/core.py:64: UserWarning:

Explicitly requested dtype int64 requested in asarray is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.



In [25]:
import tensorflow as tf

token_ids = tf.constant([101, 202, 202, 303, 404, 404, 505, 102, 606, 606, 707])
mask = tf.concat([[True], token_ids[1:] != token_ids[:-1]], axis=0)

In [26]:
token_ids[1:]

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([202, 202, 303, 404, 404, 505, 102, 606, 606, 707], dtype=int32)>

In [27]:
token_ids[:-1]

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([101, 202, 202, 303, 404, 404, 505, 102, 606, 606], dtype=int32)>

In [28]:
token_ids[1:] != token_ids[:-1]

<tf.Tensor: shape=(10,), dtype=bool, numpy=
array([ True, False,  True,  True, False,  True,  True,  True, False,
        True])>

In [29]:
#字符处理
def get_tokens(words, seq_len, packer):
    # Tokenize input  将单词列表转换成id
    token_words = tf.expand_dims( 
        tokenizer(words), axis=-1
    )  # ex: (words) ["It's", "a", "cat"] ->  (token_words) [[1, 2], [3], [4]]
    # 将id列表重塑为一列
    tokens = tf.reshape(
        token_words, [-1]
    )  # ex: (token_words) [[1, 2], [3], [4]] -> (tokens) [1, 2, 3, 4]
    # Pad tokens
    tokens = packer(tokens)[0][:seq_len] #进行特殊字符标记
    inputs = {"token_ids": tokens, "padding_mask": tokens != 0} #确定input数组
    return inputs, tokens, token_words 


def get_token_ids(token_words):
    # Get word indices   单词索引
    word_ids = tf.range(tf.shape(token_words)[0])
    # Get size of each word  实验map_fn统计长度维度的length 维度自动确定为一列
    word_size = tf.reshape(tf.map_fn(lambda word: tf.shape(word)[0:1], token_words), [-1])
    # Repeat word_id with size of word to get token_id  # 将单词的id重复对应wordsize次数
    token_ids = tf.repeat(word_ids, word_size)   
    return token_ids


def get_token_labels(word_labels, token_ids, seq_len):
    # Create token_labels from word_labels ->  alignment  将word和token_ids进行对齐
    token_labels = tf.gather(word_labels, token_ids) 
    # Only label the first token of a given word and assign -100 to others  
    mask = tf.concat([[True], token_ids[1:] != token_ids[:-1]], axis=0) # 标记每个单词开始的位置
    token_labels = tf.where(mask, token_labels, -100) #每个单词除去开头的id，后续被mask
    # Truncate to max sequence length 
    token_labels = token_labels[: seq_len - 2]  # -2 for special tokens ([CLS], [SEP]) 取出最后两个特殊字符
    # Pad token_labels to align with tokens (use -100 to pad for loss/metric ignore)
    pad_start = 1  # for [CLS] token 为cls留出一个位置
    pad_end = seq_len - tf.shape(token_labels)[0] - 1  # for [SEP] and [PAD] tokens 为了sep留出最后一个位置
    token_labels = tf.pad(token_labels, [[pad_start, pad_end]], constant_values=-100) #标记pad
    return token_labels


def process_token_ids(token_ids, seq_len):
    # Truncate to max sequence length
    token_ids = token_ids[: seq_len - 2]  # -2 for special tokens ([CLS], [SEP])
    # Pad token_ids to align with tokens (use -1 to pad for later identification)
    pad_start = 1  # [CLS] token
    pad_end = seq_len - tf.shape(token_ids)[0] - 1  # [SEP] and [PAD] tokens
    token_ids = tf.pad(token_ids, [[pad_start, pad_end]], constant_values=-1)
    return token_ids


def process_data(seq_len=720, has_label=True, return_ids=False):
    # To add spetical tokens: [CLS], [SEP], [PAD]   标记特殊符号
    packer = keras_nlp.layers.MultiSegmentPacker(
        start_value=tokenizer.cls_token_id,
        end_value=tokenizer.sep_token_id,
        sequence_length=seq_len,
    )

    def process(x):
        # Generate inputs from tokens 转换为tokenize列表
        inputs, tokens, words_int = get_tokens(x["words"], seq_len, packer) 
        # Generate token_ids for maping tokens to words
        token_ids = get_token_ids(words_int)  # 单词token重复对应字数次
        if has_label:
            # Generate token_labels from word_labels  获取单词分割后的序列以及pad标记
            token_labels = get_token_labels(x["labels"], token_ids, seq_len)
            return inputs, token_labels  #返回标记words和 tokens
        elif return_ids:
            # Pad token_ids to align with tokens
            token_ids = process_token_ids(token_ids, seq_len)  # 直接返回没有label的pad序列
            return token_ids
        else:
            return inputs

    return process


#构造数据集
def build_dataset(words, labels=None, return_ids=False, batch_size=4,
                  seq_len=512, shuffle=False, cache=True, drop_remainder=True):
    AUTO = tf.data.AUTOTUNE  #自动选择数据并行数量

    slices = {"words": tf.ragged.constant(words)} #将不同长度word文本转换成一个Ragged Tensor
    if labels is not None:
        slices.update({"labels": tf.ragged.constant(labels)}) #将不同长度label文本转换成一个Ragged Tensor

    ds = tf.data.Dataset.from_tensor_slices(slices) # 从slices变为数据集
    ds = ds.map(process_data(seq_len=seq_len,
                             has_label=labels is not None, 
                             return_ids=return_ids), num_parallel_calls=AUTO) # apply processing
    ds = ds.cache() if cache else ds  # cache dataset 数据加载到内存
    if shuffle: # shuffle dataset
        ds = ds.shuffle(1024, seed=CFG.seed)  #使用一个固定大小的缓冲区（1024个元素）来随机打乱数据
        opt = tf.data.Options()  # 确保多线程中的实验不确定性问题 
        opt.experimental_deterministic = False  # 此处表示当数据集不能被bs整除时，保留最后一个不完整批次
        ds = ds.with_options(opt)  #加载配置
    ds = ds.batch(batch_size, drop_remainder=drop_remainder)  # batch dataset 数据集划分batch
    ds = ds.prefetch(AUTO)  # prefetch next batch  自动获取下一个批次
    return ds

In [30]:
train_ds = build_dataset(train_words, train_labels,  batch_size=CFG.train_batch_size,
                         seq_len=CFG.train_seq_len, shuffle=True)

valid_ds = build_dataset(valid_words, valid_labels, batch_size=CFG.train_batch_size, 
                         seq_len=CFG.train_seq_len, shuffle=False)

In [31]:
inp, tar = next(iter(valid_ds))
print("# Input:\n",inp); print("\n# Labels:\n",tar)

# Input:
 {'token_ids': <tf.Tensor: shape=(16, 1024), dtype=int32, numpy=
array([[    1, 28525,   877, ...,     0,     0,     0],
       [    1, 45730,   377, ...,     0,     0,     0],
       [    1,  8489,  7933, ...,     0,     0,     0],
       ...,
       [    1,  6348, 28525, ...,  1068,  3955,     2],
       [    1,  3780, 13942, ...,     0,     0,     0],
       [    1, 45730,   377, ...,  6568,   366,     2]], dtype=int32)>, 'padding_mask': <tf.Tensor: shape=(16, 1024), dtype=bool, numpy=
array([[ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ...,  True,  True,  True]])>}

# Labels:
 tf.Tensor(
[[-100   12   12 ... -100 -100 -100]
 [-100   12   12 ... -100 -100 -100]
 [-100   12 -100 ... -100 -100 -100]
 ...
 [-100 

In [32]:
#交叉熵定义
class CrossEntropy(keras.losses.SparseCategoricalCrossentropy):  # 交叉熵实现
    def __init__(self, ignore_class=-100, reduction=None, **args):
        super().__init__(reduction=reduction, **args)
        self.ignore_class = ignore_class

    def call(self, y_true, y_pred):
        y_true = ops.reshape(y_true, [-1])
        y_pred = ops.reshape(y_pred, [-1, CFG.num_labels])
        loss = super().call(y_true, y_pred)
        if self.ignore_class is not None:
            valid_mask = ops.not_equal(
                y_true, ops.cast(self.ignore_class, y_pred.dtype)
            )
            loss = ops.where(valid_mask, loss, 0.0)
            loss = ops.sum(loss)
            loss /= ops.maximum(ops.sum(ops.cast(valid_mask, loss.dtype)), 1)
        else:
            loss = ops.mean(loss)
        return loss

In [33]:
# F beta 评价指标
class FBetaScore(keras.metrics.FBetaScore):
    def __init__(self, ignore_classes=[-100, 12], average="micro", beta=5.0,
                 name="f5_score", **args):
        super().__init__(beta=beta, average=average, name=name, **args)
        self.ignore_classes = ignore_classes or []

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = ops.convert_to_tensor(y_true, dtype=self.dtype)
        y_pred = ops.convert_to_tensor(y_pred, dtype=self.dtype)
        
        y_true = ops.reshape(y_true, [-1])
        y_pred = ops.reshape(y_pred, [-1, CFG.num_labels])
            
        valid_mask = ops.ones_like(y_true, dtype=self.dtype)
        if self.ignore_classes:
            for ignore_class in self.ignore_classes:
                valid_mask &= ops.not_equal(y_true, ops.cast(ignore_class, y_pred.dtype))
        valid_mask = ops.expand_dims(valid_mask, axis=-1)
        
        y_true = ops.one_hot(y_true, CFG.num_labels)
        
        if not self._built:
            self._build(y_true.shape, y_pred.shape)

        threshold = ops.max(y_pred, axis=-1, keepdims=True)
        y_pred = ops.logical_and(
            y_pred >= threshold, ops.abs(y_pred) > 1e-9
        )

        y_pred = ops.cast(y_pred, dtype=self.dtype)
        y_true = ops.cast(y_true, dtype=self.dtype)
        
        tp = ops.sum(y_pred * y_true * valid_mask, self.axis)
        fp = ops.sum(y_pred * (1 - y_true) * valid_mask, self.axis)
        fn = ops.sum((1 - y_pred) * y_true * valid_mask, self.axis)
            
        self.true_positives.assign_add(tp)
        self.false_positives.assign_add(fp)
        self.false_negatives.assign_add(fn)

In [34]:
# Build Token Classification model  建立基于tokens分类的模型管道
backbone = keras_nlp.models.DebertaV3Backbone.from_preset(
    CFG.preset,
)
out = backbone.output
out = keras.layers.Dense(CFG.num_labels, name="logits")(out) #修改输出头
out = keras.layers.Activation("softmax", dtype="float32", name="prediction")(out) #加一层分类激活函数
model = keras.models.Model(backbone.input, out) #建立管道

# Compile model for optimizer, loss and metric
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=2e-5), #
    loss=CrossEntropy(),
    metrics=[FBetaScore()],
)

# Summary of the model architecture
model.summary()

Attaching 'config.json' from model 'keras/deberta_v3/keras/deberta_v3_small_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/deberta_v3/keras/deberta_v3_small_en/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/deberta_v3/keras/deberta_v3_small_en/2' to your Kaggle notebook...


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ token_ids           │ (None, None)      │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ token_embedding     │ (None, None, 768) │ 98,380… │ token_ids[0][0]      │
│ (ReversibleEmbeddi… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ embeddings_layer_n… │ (None, None, 768) │   1,536 │ token_embedding[0][… │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ embeddings_dropout  │ (None, None, 768) │       0 │ embeddings_layer_no… │
│ (Dropout)           │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ padding_mask        │ (None, None)      │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ rel_embedding       │ (None, 512, 768)  │ 394,752 │ embeddings_dropout[… │
│ (RelativeEmbedding) │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ disentangled_atten… │ (None, None, 768) │ 7,087,… │ embeddings_dropout[… │
│ (DisentangledAtten… │                   │         │ padding_mask[0][0],  │
│                     │                   │         │ rel_embedding[0][0]  │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ disentangled_atten… │ (None, None, 768) │ 7,087,… │ disentangled_attent… │
│ (DisentangledAtten… │                   │         │ padding_mask[0][0],  │
│                     │                   │         │ rel_embedding[0][0]  │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ disentangled_atten… │ (None, None, 768) │ 7,087,… │ disentangled_attent… │
│ (DisentangledAtten… │                   │         │ padding_mask[0][0],  │
│                     │                   │         │ rel_embedding[0][0]  │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ disentangled_atten… │ (None, None, 768) │ 7,087,… │ disentangled_attent… │
│ (DisentangledAtten… │                   │         │ padding_mask[0][0],  │
│                     │                   │         │ rel_embedding[0][0]  │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ disentangled_atten… │ (None, None, 768) │ 7,087,… │ disentangled_attent… │
│ (DisentangledAtten… │                   │         │ padding_mask[0][0],  │
│                     │                   │         │ rel_embedding[0][0]  │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ disentangled_atten… │ (None, None, 768) │ 7,087,… │ disentangled_attent… │
│ (DisentangledAtten… │                   │         │ padding_mask[0][0],  │
│                     │                   │         │ rel_embedding[0][0]  │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ logits (Dense)      │ (None, None, 13)  │   9,997 │ disentangled_attent… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ prediction          │ (None, None, 13)  │       0 │ logits[0][0]         │
│ (Activation)        │                   │         │                      │
└─────────────────────┴───────────────────┴─────────┴──────────────────────┘

 Total params: 141,314,317 (539.07 MB)

 Trainable params: 141,314,317 (539.07 MB)

 Non-trainable params: 0 (0.00 B)

In [35]:
import math

#详细定义学习率schedule
def get_lr_callback(batch_size=8, mode='cos', epochs=10, plot=False):
    lr_start, lr_max, lr_min = 6e-6, 2.5e-6 * batch_size, 1e-6
    lr_ramp_ep, lr_sus_ep, lr_decay = 3, 0, 0.75

    def lrfn(epoch):  # Learning rate update function
        if epoch < lr_ramp_ep: lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
        elif epoch < lr_ramp_ep + lr_sus_ep: lr = lr_max
        elif mode == 'exp': lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
        elif mode == 'step': lr = lr_max * lr_decay**((epoch - lr_ramp_ep - lr_sus_ep) // 2)
        elif mode == 'cos':
            decay_total_epochs, decay_epoch_index = epochs - lr_ramp_ep - lr_sus_ep + 3, epoch - lr_ramp_ep - lr_sus_ep
            phase = math.pi * decay_epoch_index / decay_total_epochs
            lr = (lr_max - lr_min) * 0.5 * (1 + math.cos(phase)) + lr_min
        return lr

    if plot:  # Plot lr curve if plot is True
        fig = px.line(x=np.arange(epochs),
                      y=[lrfn(epoch) for epoch in np.arange(epochs)], 
                      title='LR Scheduler',
                      markers=True,
                      labels={'x': 'epoch', 'y': 'lr'})
        fig.update_layout(
            yaxis = dict(
                showexponent = 'all',
                exponentformat = 'e'
            )
        )
        fig.show()

    return keras.callbacks.LearningRateScheduler(lrfn, verbose=False)  # Create lr callback

In [36]:
lr_cb = get_lr_callback(CFG.train_batch_size, mode=CFG.lr_mode, plot=True)

In [37]:
#建立数据以及参数，模型训练的pipline
if CFG.train:
    history = model.fit(
        train_ds,
        validation_data=valid_ds,
        epochs=CFG.epochs,
        callbacks=[lr_cb],
        verbose=1,
    )
else:
    model.load_weights("/kaggle/input/pii-data-detection-ckpt-ds/model.weights.h5")

Epoch 1/6
340/340 ━━━━━━━━━━━━━━━━━━━━ 630s 2s/step - f5_score: 0.0254 - loss: 0.4733 - val_f5_score: 0.0111 - val_loss: 0.0044 - learning_rate: 6.0000e-06
Epoch 2/6
340/340 ━━━━━━━━━━━━━━━━━━━━ 528s 2s/step - f5_score: 0.0930 - loss: 0.0031 - val_f5_score: 0.5498 - val_loss: 0.0013 - learning_rate: 1.7333e-05
Epoch 3/6
340/340 ━━━━━━━━━━━━━━━━━━━━ 523s 2s/step - f5_score: 0.5493 - loss: 0.0014 - val_f5_score: 0.8506 - val_loss: 7.0660e-04 - learning_rate: 2.8667e-05
Epoch 4/6
340/340 ━━━━━━━━━━━━━━━━━━━━ 515s 2s/step - f5_score: 0.7961 - loss: 6.2621e-04 - val_f5_score: 0.9114 - val_loss: 5.2849e-04 - learning_rate: 4.0000e-05
Epoch 5/6
340/340 ━━━━━━━━━━━━━━━━━━━━ 514s 2s/step - f5_score: 0.9310 - loss: 2.5756e-04 - val_f5_score: 0.8469 - val_loss: 4.4582e-04 - learning_rate: 3.0250e-05
Epoch 6/6
340/340 ━━━━━━━━━━━━━━━━━━━━ 511s 2s/step - f5_score: 0.9630 - loss: 1.3233e-04 - val_f5_score: 0.8967 - val_loss: 4.0003e-04 - learning_rate: 2.2938e-05


In [38]:
#构建验证集合
# Build Validation dataloader with "infer_seq_len"
valid_ds = build_dataset(valid_words, valid_labels, return_ids=False, batch_size=CFG.infer_batch_size,
                        seq_len=CFG.infer_seq_len, shuffle=False, cache=False)

In [39]:
model.evaluate(valid_ds,return_dict=True,verbose=0)

{'f5_score': 0.8926653861999512, 'loss': 0.0004159419040661305}

In [41]:
test_data = json.load(open(f"{BASE_PATH}/test.json"))
#增补末尾样本，对齐batch数量
need_samples = len(devices) - len(test_data) % len(devices)
for _ in range(need_samples):
    test_data.append(test_data[-1])

#创建容器
test_words = np.empty(len(test_data),dtype=object)
test_docs = np.empty(len(test_data),dtype=np.int32)

#将字段赋值到数组中
for i,x in tqdm(enumerate(test_data),total=len(test_data)):
    test_words[i] = np.array(x['tokens'])
    test_docs[i] = x['document']

#测试集构建    
id_ds = build_dataset(test_words,return_ids=True, batch_size=len(test_words), 
                        seq_len=CFG.infer_seq_len, shuffle=False, cache=False, drop_remainder=False)
#只取出测试集的token ids
test_token_ids = ops.convert_to_numpy([ids for ids in iter(id_ds)][0])

# Build test dataloader 
test_ds = build_dataset(test_words, return_ids=False, batch_size=CFG.infer_batch_size,
                        seq_len=CFG.infer_seq_len, shuffle=False, cache=False, drop_remainder=False)

  0%|          | 0/12 [00:00<?, ?it/s]

In [42]:
test_preds = model.predict(test_ds,verbose=1)

test_preds

3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step


array([[[1.3834994e-01, 5.3652093e-02, 2.8588483e-02, ...,
         4.8667604e-01, 6.1542470e-02, 1.7658062e-02],
        [3.1376530e-07, 2.9160586e-07, 4.2788447e-06, ...,
         1.1078923e-07, 9.7580390e-08, 9.9998879e-01],
        [3.9509092e-07, 1.2876927e-07, 2.3466221e-06, ...,
         2.9619855e-07, 9.5506479e-08, 9.9999070e-01],
        ...,
        [2.6591924e-05, 9.2182614e-05, 1.2974467e-04, ...,
         5.1558491e-05, 2.1077069e-05, 9.9635363e-01],
        [2.6591924e-05, 9.2182614e-05, 1.2974467e-04, ...,
         5.1558491e-05, 2.1077069e-05, 9.9635363e-01],
        [2.6591924e-05, 9.2182614e-05, 1.2974467e-04, ...,
         5.1558491e-05, 2.1077069e-05, 9.9635363e-01]],

       [[1.3684441e-01, 5.3406183e-02, 2.8453974e-02, ...,
         4.8091015e-01, 6.4893723e-02, 1.9692658e-02],
        [6.9254870e-04, 2.2013328e-05, 9.6806484e-01, ...,
         9.5776003e-04, 3.9789148e-04, 1.7118210e-02],
        [2.9730154e-04, 7.9593847e-05, 4.0494069e-05, ...,
         2.891

In [43]:
test_preds.shape

(12, 2000, 13)

In [44]:
#取最后一个维度的预测结果
test_preds = np.argmax(test_preds,axis=-1)
test_preds

array([[10, 12, 12, ..., 12, 12, 12],
       [10,  2,  8, ..., 12, 12, 12],
       [10, 12, 12, ..., 12, 12, 12],
       ...,
       [10, 12, 12, ..., 12, 12, 12],
       [10, 12, 12, ..., 12, 12, 12],
       [10, 12, 12, ..., 12, 12, 12]])

In [45]:
#移除增补的重复数据
test_docs = test_docs[:-need_samples]
test_token_ids = test_token_ids[:-need_samples]
test_preds = test_preds[:-need_samples]
test_words = test_words[:-need_samples]

In [46]:
#掩码筛选数据
document_list = []
token_id_list = []
label_id_list = []
token_list = []

for doc, token_ids, preds, tokens in tqdm(
    zip(test_docs, test_token_ids, test_preds, test_words), total=len(test_words)
):
    # Create mask for filtering
    #筛选非token 开始字符、预测有值、标记特殊字符的标签
    mask1 = np.concatenate(([True], token_ids[1:] != token_ids[:-1])) # ignore non-start tokens of a word
    mask2 = (preds != 12) # ignore `O` (BIO format) label -> 12 (integer format) label
    mask3 = (token_ids != -1)  # ignore [CLS], [SEP], and [PAD] tokens
    mask = (mask1 & mask2 & mask3) # merge filters
    
    # Apply filter
    #进行筛选
    token_ids = token_ids[mask]
    preds = preds[mask]

     # Store prediction if number of tokens is not zero
    if len(token_ids):
        token_list.extend(tokens[token_ids])
        document_list.extend([doc] * len(token_ids))
        token_id_list.extend(token_ids)
        label_id_list.extend(preds)

  0%|          | 0/10 [00:00<?, ?it/s]

In [47]:
# 创建表格
pred_df = pd.DataFrame(
    {
        "document": document_list,
        "token": token_id_list,
        "label_id": label_id_list,
        "token_string": token_list,
    }
)
pred_df = pred_df.rename_axis("row_id").reset_index() # add `row_id` column
pred_df["label"] = pred_df.label_id.map(CFG.id2label) # map integer label to BIO format label
pred_df.head(10) 

,row_id,document,token,label_id,token_string,label
0,0,7,9,2,Nathalie,B-NAME_STUDENT
1,1,7,10,8,Sylla,I-NAME_STUDENT
2,2,7,482,2,Nathalie,B-NAME_STUDENT
3,3,7,483,8,Sylla,I-NAME_STUDENT
4,4,7,741,2,Nathalie,B-NAME_STUDENT
5,5,7,742,8,Sylla,I-NAME_STUDENT
6,6,10,0,2,Diego,B-NAME_STUDENT
7,7,10,1,8,Estrada,I-NAME_STUDENT
8,8,10,464,2,Diego,B-NAME_STUDENT
9,9,10,465,8,Estrada,I-NAME_STUDENT


In [48]:
sub_df = pred_df.drop(columns=["token_string", "label_id"]) # remove extra columns
sub_df.to_csv("submission.csv", index=False)